In [15]:
pip install geopy


[notice] A new release of pip is available: 23.3.1 -> 24.3.1
[notice] To update, run: pip3 install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [12]:
# import pyarrow.parquet as pq
import pandas as pd
import json
from geopy.geocoders import Nominatim
from tqdm.auto import tqdm

geolocator = Nominatim(user_agent="gtfs_processor", timeout=2000)

with open('poi.json') as fp:
    data = json.load(fp)

# df = pd.read_json('poi.json')
# df

df = []
for item in tqdm(data):

    latitude = item['content']['poiData']['location']['coordinate']['latitude']
    longitude = item['content']['poiData']['location']['coordinate']['longitude']

    location = geolocator.reverse((latitude, longitude), exactly_one=True)
    
    if location and 'osm_id' in location.raw:
        osm_id =location.raw['osm_id']
    else:
        osm_id = 'na'
    
    new_json_data = {
        "id": item['_id'],
        "latitude": latitude,
        "longitude": longitude,
        "name": item['content']['objData']['name']['IT'],
        "type": item['content']['objData']['category'],
        "osm_id": osm_id,
    }
    df.append(new_json_data)
    
df = pd.DataFrame(df).to_csv('poi_and_osm.csv', index=False)

newdf = pd.read_csv('poi_and_osm.csv')

print(newdf.head())

  0%|          | 0/2893 [00:00<?, ?it/s]

                                                  id   latitude  longitude  \
0  it.trentour.domains.core.PointOfInterest.52af1...  46.144100  11.118600   
1  it.trentour.domains.core.PointOfInterest.52af1...  46.083546  11.174873   
2  it.trentour.domains.core.PointOfInterest.529ef...  46.428959  11.695014   
3  it.trentour.domains.core.PointOfInterest.529ef...  46.328877  11.601681   
4  it.trentour.domains.core.PointOfInterest.529ef...  46.393068  11.799665   

                           name         type      osm_id  
0  MASO FRANCH GOURMET E RELAIS        hotel  1219539037  
1               MASO CANTANGHEL   ristorante  1178258840  
2           Ski Stadium «Aloch»  impiantosci    27217071  
3    Stadio del Salto «Dal Ben»  stadiosalto    26821697  
4          Passo San Pellegrino  impiantosci  1223552917  
